In [201]:
from pybit import inverse_perpetual  # <-- import HTTP & WSS for inverse perp
from pybit import spot  # <-- import HTTP & WSS for spot
from pybit import HTTP  # supports inverse perp & futures, usdt perp, spot.
from pybit.spot import HTTP
from pprint import pprint
from dotenv import load_dotenv
import os
import json
import pandas as pd
from time import sleep

In [202]:
load_dotenv()

True

In [203]:
api_key_pw = os.getenv('api_key')
api_secret_pw = os.getenv('api_secret')


In [204]:
session = inverse_perpetual.HTTP(
    endpoint='https://api.bybit.com', 
    api_key= api_key_pw,
    api_secret= api_secret_pw
)
ws = inverse_perpetual.WebSocket(
    test=False,
    api_key= api_key_pw,
    api_secret= api_secret_pw
)

In [205]:
pd.DataFrame(session.latest_information_for_symbol(symbol ='BTCUSD')["result"]).keys()

Index(['symbol', 'bid_price', 'ask_price', 'last_price', 'last_tick_direction',
       'prev_price_24h', 'price_24h_pcnt', 'high_price_24h', 'low_price_24h',
       'prev_price_1h', 'mark_price', 'index_price', 'open_interest',
       'countdown_hour', 'turnover_24h', 'volume_24h', 'funding_rate',
       'predicted_funding_rate', 'next_funding_time',
       'predicted_delivery_price', 'total_turnover', 'total_volume',
       'delivery_fee_rate', 'delivery_time', 'price_1h_pcnt', 'open_value'],
      dtype='object')

In [206]:
session.orderbook(symbol='BTCUSD')['result'][0]

{'symbol': 'BTCUSD', 'price': '16825.5', 'side': 'Buy', 'size': 273276}

In [207]:
df = pd.DataFrame(session.query_symbol()['result'])
df.sample()

,name,alias,status,base_currency,quote_currency,price_scale,taker_fee,maker_fee,funding_interval,leverage_filter,price_filter,lot_size_filter
43,BTCUSDT,BTCUSDT,Trading,BTC,USDT,2,0.0006,0.0001,480,"{'min_leverage': 1, 'max_leverage': 100, 'leve...","{'min_price': '0.50', 'max_price': '999999.00'...","{'max_trading_qty': 420, 'min_trading_qty': 0...."


In [230]:
btc = df.loc[(df.name=='BTCUSDT')]
btc

,name,alias,status,base_currency,quote_currency,price_scale,taker_fee,maker_fee,funding_interval,leverage_filter,price_filter,lot_size_filter
43,BTCUSDT,BTCUSDT,Trading,BTC,USDT,2,0.0006,0.0001,480,"{'min_leverage': 1, 'max_leverage': 100, 'leve...","{'min_price': '0.50', 'max_price': '999999.00'...","{'max_trading_qty': 420, 'min_trading_qty': 0...."


In [231]:
def clean(dataframe):
    dataframe = btc.explode('price_filter').reset_index(drop= True)
    #dataframe = btc.explode("leverage_filter").reset_index(drop= True)
    dataframe = pd.concat([btc, btc["price_filter"].apply(pd.Series)], axis=1).reset_index(drop=True)
    #dataframe = pd.concat([btc, btc['leverage_filter'].apply(pd.Series)], axis=1).reset_index(drop=True)
    return dataframe

In [232]:
btc = clean(btc)
btc

,name,alias,status,base_currency,quote_currency,price_scale,taker_fee,maker_fee,funding_interval,leverage_filter,price_filter,lot_size_filter,min_price,max_price,tick_size
0,BTCUSDT,BTCUSDT,Trading,BTC,USDT,2,0.0006,0.0001,480,"{'min_leverage': 1, 'max_leverage': 100, 'leve...","{'min_price': '0.50', 'max_price': '999999.00'...","{'max_trading_qty': 420, 'min_trading_qty': 0....",0.50,999999.00,0.50


In [212]:
btc

,name,alias,status,base_currency,quote_currency,price_scale,taker_fee,maker_fee,funding_interval,leverage_filter,price_filter,lot_size_filter
43,BTCUSDT,BTCUSDT,Trading,BTC,USDT,2,0.0006,0.0001,480,"{'min_leverage': 1, 'max_leverage': 100, 'leve...","{'min_price': '0.50', 'max_price': '999999.00'...","{'max_trading_qty': 420, 'min_trading_qty': 0...."


In [235]:
from pybit import spot
session_unauth = spot.HTTP(
    endpoint="https://api-testnet.bybit.com"
)
print(session_unauth.last_traded_price(
    symbol="BTCUSDT"
))

{'ret_code': 0, 'ret_msg': '', 'result': {'symbol': 'BTCUSDT', 'price': '16929.09'}, 'ext_info': None, 'ext_code': None}
